In [31]:
# Import dependencies
import subprocess
import json
from dotenv import load_dotenv
import os
from constants import *
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from web3 import Web3
from web3.middleware import geth_poa_middleware
from eth_account import Account

In [32]:
# Load and set environment variables
load_dotenv('mnemonics.env')
mnemonic=os.getenv("BIP39_MNEMONIC")

w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [33]:
print(mnemonic)

proof calm dream lion leg again similar delay clerk twice brown island


In [34]:
# Create a function called `derive_wallets`

def derive_wallets(coin,mnemonic=mnemonic,depths=3):
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{mnemonic}" --cols=all --coin={coin} --numderive={depths} --format=json'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

In [35]:
derive_wallets(coin=ETH)

[{'path': "m/44'/60'/0'/0/0",
  'address': '0x72FaD3E6dbF9922f463aA43afd94F725345206B7',
  'xprv': 'xprvA2ZNc6Df5cM63pDwTjg5jJAGuVcDvJ8FbRnFv7yejzKhGF1hRVDdcnTPNd3SC9wC9TAeoZMDEk2BJdpATMmjBkDHCgZMqn9hQAfTHxPxR74',
  'xpub': 'xpub6FYj1bkYuyuPGJJQZmD66S71TXSiKkr6xehriWPGJKrg93Lqy2XtAamsDtNeWZzEcz9YJyxiFjtR5FpoYCxCuthDi5BXQZVSpC3xvtCNSpc',
  'privkey': '0x32bc3836c004e4cc75c047f516a3a0471071414abcf122081763b94a5413f83e',
  'pubkey': '025a594b0efac51d44a9fea855905c558232673dc3fe3cf4099250a71fffacdd6c',
  'pubkeyhash': '3b88b50cf4328d927cb61e175b759e85bff4d93b',
  'index': 0},
 {'path': "m/44'/60'/0'/0/1",
  'address': '0x4B18Ab7743ef77bC72A97b73E62354fAb59fc593',
  'xprv': 'xprvA2ZNc6Df5cM673Lq3VjJYcDNx3XgrtnaUzM6N4WJqyYfn4XbvXw2nWrV25jkWJmoGbajACMkMDm8HoiPnRDohUjgykDjh373naqz7PQMnEW',
  'xpub': 'xpub6FYj1bkYuyuPKXRJ9XGJukA7W5NBGMWRrDGhASuvQK5eerrkU5FHLKAxsMbdGTTUoZfBCipixyWcf2jsXUAZBUJ6JvDv1mTifrudrGQNCHE',
  'privkey': '0x07e98c624c3151299de8c4fdbb28466e160b247a8379e3459b5295bcb0ce5201',

In [36]:
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {
    BTC:derive_wallets(BTC,mnemonic),
    BTCTEST:derive_wallets(BTCTEST,mnemonic),
    ETH:derive_wallets(ETH,mnemonic)
}

In [37]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin,privkey):
    if coin == ETH:
        return Account.privateKeyToAccount(privkey)
    elif coin == BTCTEST:
        return PrivateKeyTestnet(privkey)
    else:
        return privkeybtc(privkey)

In [44]:
account = priv_key_to_account(ETH,'0x32bc3836c004e4cc75c047f516a3a0471071414abcf122081763b94a5413f83e')
account.address

'0x72FaD3E6dbF9922f463aA43afd94F725345206B7'

In [45]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(acct, recipient, amt):
    gasest = w3.eth.estimateGas(
        {"from": acct.address, "to": recipient, "value": amt}
    )
    return {
        "chainId": 666, 
        "from": acct.address,
        "to": recipient,
        "value": amt,
        "gasPrice": w3.eth.gasPrice,
        "gas": gasest,
        "nonce": w3.eth.getTransactionCount(acct.address),
    }

In [46]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(acct, recipient, amt):
    tx = create_tx(acct, recipient, amt)
    signed_tx = acct.sign_transaction(tx)
    output = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
    print(output.hex())
    return output.hex()

In [51]:
send_tx(account,"0x30b0EE600e253762d663aE40eFB5E17197AdA440",10000000000)

0x9272626fa6e6a07408a1dd0812749ddcfd160d2c24732d7e8cbd3016c1434e46


'0x9272626fa6e6a07408a1dd0812749ddcfd160d2c24732d7e8cbd3016c1434e46'

In [59]:
w3.eth.get_balance(account.address)

9999999998739990000000000

In [60]:
derive_wallets(coin=BTCTEST)

[{'path': "m/44'/1'/0'/0/0",
  'address': 'musFVHfFMAu4bmeTwTB76FnJRxJTpRtZvN',
  'xprv': 'tprv8kokjCiCFAyQuDyrUdY7ui1MKasZkBmravt71UAJ9UEkUXhq29yYo7bKBmTvxhaVXRBqJwh3q8ewu2iCGBKiDQxAeUuTxpc7h9uc9wRNxEd',
  'xpub': 'tpubDHVnsckSPYf5nh1eNHCiK7fTtcPVuWxmAEUtHzCbZk39K1xbeYo8ycDBMw1FHm65RuGLTBLWCTrkik9BgC2FDfv6FvcWjcAYJPrgnJAxxdC',
  'privkey': 'cNLGwNwcg4cCSGnemXZWfCgeppmnTcrCa6SNUEKqo5kexUSZDkZt',
  'pubkey': '036bf6a068691b18602f8af77b32cd581d197c643df27aa59ca4b14bf7344f8de3',
  'pubkeyhash': '9d686eca90adf2ac6e7bf2dc847fb6b15a41a487',
  'index': 0},
 {'path': "m/44'/1'/0'/0/1",
  'address': 'myKQSEjyp4qJgN6q8AehMbhLZVqDnZ3ZSL',
  'xprv': 'tprv8kokjCiCFAyQvdNCwcGfm5TCuR7knfB5iadNZkumh4ajyat2MWKemFeE9nJZ6F2EZmtoU1n4PiSFeCGhovABLYKsyZh9PCKHd8BAeVKJp8N',
  'xpub': 'tpubDHVnsckSPYf5p6PzqFwGAV7KUSdgwzMzHtE9rGx57LP8p58nyu9EwkG6KuvfianMnykqDCbtwvRafe76bmjbCYoL6WaZS1R8n1BP1sUfJ6z',
  'privkey': 'cTJgdYcXWqnKXSMHjrLvNhAJXxE3CeJZwPXW8jSW5Ue6LVgF6csG',
  'pubkey': '0303d59da7ef3fecc8be850ae111fcee